In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [35]:
train.head()

,id,gravity,ph,osmo,cond,urea,calc,target
0,0,1.013,6.19,443,14.8,124,1.45,0
1,1,1.025,5.40,703,23.6,394,4.18,0
2,2,1.009,6.13,371,24.5,159,9.04,0
3,3,1.021,4.91,442,20.8,398,6.63,1
4,4,1.021,5.53,874,17.8,385,2.21,1


In [36]:
test.head()

,id,gravity,ph,osmo,cond,urea,calc
0,414,1.017,5.24,345,11.5,152,1.16
1,415,1.020,5.68,874,29.0,385,3.46
2,416,1.024,5.36,698,19.5,354,13.00
3,417,1.020,5.33,668,25.3,252,3.46
4,418,1.011,5.87,567,29.0,457,2.36


In [9]:
X = train.iloc[:, 1:-1]
y = train['target']

In [10]:
target = test[X.columns]

In [21]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import auc, roc_curve

In [18]:
kf = KFold(n_splits = 5, random_state = 42, shuffle = True)

In [29]:
cb_prob = np.zeros(target.shape[0])
cb_auc = 0
for i, (tr_idx, val_idx) in enumerate(kf.split(X)) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    cb = CatBoostClassifier(random_state = 42, max_depth = 6, learning_rate = 0.01, n_estimators = 1000, eval_metric = 'AUC')
    cb.fit(tr_x, tr_y, eval_set = [(tr_x, tr_y), (val_x, val_y)], early_stopping_rounds = 100, verbose = 200)
    
    val_pred = cb.predict_proba(val_x)[:, 1]
    fpr, tpr, threshold = roc_curve(val_y, val_pred)
    val_auc = auc(fpr, tpr)
    cb_auc += val_auc / kf.n_splits
    print(f'{i + 1} Fold AUC : {val_auc}')
    
    fold_pred = cb.predict_proba(target)[:, 1] / kf.n_splits
    cb_prob += fold_pred
print(f'\n{cb.__class__.__name__} AVG of AUC : {cb_auc}')

0:	test: 0.8191411	test1: 0.8146199	best: 0.8146199 (0)	total: 2.04ms	remaining: 2.03s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.8643274854
bestIteration = 25

Shrink model to first 26 iterations.
1 Fold AUC : 0.8643274853801169
0:	test: 0.8065832	test1: 0.7192671	best: 0.7192671 (0)	total: 877us	remaining: 877ms
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.8209219858
bestIteration = 39

Shrink model to first 40 iterations.
2 Fold AUC : 0.8209219858156028
0:	test: 0.8445916	test1: 0.6830303	best: 0.6830303 (0)	total: 813us	remaining: 813ms
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6975757576
bestIteration = 15

Shrink model to first 16 iterations.
3 Fold AUC : 0.6975757575757576
0:	test: 0.8305339	test1: 0.8039044	best: 0.8039044 (0)	total: 643us	remaining: 643ms
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.8630536131
bestIteration = 84

Shrink model to first 85 iterations.
4 Fold AUC 

In [30]:
submission = pd.read_csv('sample_submission.csv')

In [32]:
submission['target'] = cb_prob

In [34]:
submission.to_csv('cb.csv', index = False)